In [ ]:
Install-Module pskit -force

In [ ]:
function ConvertTo-MarkdownTable {
    param($targetData)

    $names = $targetData[0].psobject.Properties.name     

    $all = @()
    1..$names.count | foreach {
        if($_ -eq $names.count) {
            $all += '|'
        } else {
            $all += '|---'
        }
    }

    $result = foreach($record in $targetData) {
        $inner=@()
        foreach($name in $names) {        
            $inner+=$record.$name
        }        
        '|' + ($inner -join '|') + '|' + "`n"
    }
    
@"
$('|' + ($names -join '|') + '|')
$($all)
$($result)
"@ | ConvertFrom-Markdown | % html | Get-HtmlContent | Out-Display
}

In [ ]:
$date=(Get-Date).AddDays(-1).ToString('MM-dd-yyyy')
$url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{0}.csv' -f $date

$data = ConvertFrom-Csv (irm $url)
$table = New-LookupTable $data Country/Region

$result = foreach ($key in $table.keys) {
    [pscustomobject][ordered]@{
        Key = $key
        TotalConfirmed=($table.$key| measure confirmed -Sum).Sum
    }
}


In [ ]:
$data = ($result | sort-object TotalConfirmed -Descending | select -First 10 -skip 3)

ConvertTo-MarkdownTable $data

$ConfirmedSeries = [Graph.Bar]@{
    name = "TotalConfirmed"
    x = $data.Key
    y = $data.TotalConfirmed
}

$chart = $ConfirmedSeries | New-PlotlyChart -Title "Total Confirmed - COVID-19"
Out-Display $chart